# eCommerce Events History in Cosmetics Shop Data Processing
######  ※因為原始資料檔案太大，因此將Python程式分為資料預處理和資料分析兩部分，本檔案為資料預處理的一部分。  
######  ※此為使用五個月數據分析的測試集，將會使用全部五個月的數據來進行集群分析，供比對顧客集群結果使用。  
此專案將針對化妝品電商數據進行分析，並且針對顧客進行分群研究，找出其中的商業價值。  
此數據集是一家中型化妝品電商在2019年10月至2020年2月間的各種消費者行為數據。一筆數據代表一個事件，每個事件都類似於產品和用戶之間的多對多關係。

1. 資料預處理
2. 探索性資料分析
3. 重要經營指標分析
4. RSFM集群分析
5. 集群結果分析
6. Tableau視覺化報表
7. 測試集集群對照
8. 結論

Dataset: https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop , from  REES46 Marketing Platform.

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import os 
pd.set_option('display.float_format', lambda x: '%.3f' % x) #禁用科學記號不然數字會很難看

## Loading Data

In [2]:
data1 = pd.read_csv('./ecommerce/2019-Oct.csv')

In [3]:
data1.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,NaN,runail,2.620,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,NaN,runail,2.620,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,NaN,lovely,13.480,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,NaN,runail,2.620,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,NaN,lovely,0.560,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9


In [4]:
files= [file for file in os.listdir('./ecommerce')]

all_m_data= pd.DataFrame()
for file in files:
    df = pd.read_csv('./ecommerce/' + file)
    print(f"{df}'s shape: {df.shape}")
    all_m_data = pd.concat([all_m_data,df])
all_m_data.drop('user_session', axis= 1, inplace= True)
all_m_data.to_csv("alldata.csv", index= False)

                      event_time        event_type  product_id  \
0        2019-12-01 00:00:00 UTC  remove_from_cart     5712790   
1        2019-12-01 00:00:00 UTC              view     5764655   
2        2019-12-01 00:00:02 UTC              cart        4958   
3        2019-12-01 00:00:05 UTC              view     5848413   
4        2019-12-01 00:00:07 UTC              view     5824148   
...                          ...               ...         ...   
3533281  2019-12-31 23:59:39 UTC              view     5683350   
3533282  2019-12-31 23:59:46 UTC              view     5888097   
3533283  2019-12-31 23:59:51 UTC              view       59975   
3533284  2019-12-31 23:59:52 UTC              view     5775982   
3533285  2019-12-31 23:59:57 UTC              view     5635090   

                 category_id category_code      brand   price    user_id  \
0        1487580005268456287           NaN      f.o.x   6.270  576802932   
1        1487580005411062629           NaN        cnd  

In [5]:
data = pd.read_csv('alldata.csv')
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.270,576802932
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.050,412120092
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.190,494077766
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.790,348405118
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.560,576005683


## Data Cleaning

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20692840 entries, 0 to 20692839
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 1.2+ GB


In [7]:
data.nunique()

event_time       8782890
event_type             4
product_id         54571
category_id          525
category_code         12
brand                273
price               2860
user_id          1639358
dtype: int64

In [8]:
print("Duplicates : ", len(data[data.duplicated()]))

Duplicates :  1120949


可以看見數據中有100多萬筆重複值（所有欄位皆重複），這邊將重複的觀測值刪除。

In [9]:
data.drop_duplicates(inplace= True)

看一下資料的分布。

In [10]:
data.describe()

,product_id,category_id,price,user_id
count,19571891.000,19571891.000,19571891.000,19571891.000
mean,5481753.730,1554506323053204736.000,8.736,522120816.949
std,1310988.312,169466634088880352.000,19.751,87715872.193
min,3752.000,1487580004807082752.000,-79.370,465496.000
25%,5724652.000,1487580005754995456.000,2.110,483433608.000
50%,5810718.000,1487580008263189504.000,4.110,554125845.000
75%,5857952.000,1487580013522845952.000,7.140,579625275.500
max,5932595.000,2242903426784559104.000,327.780,622090237.000


發現price欄竟然有一些負值，檢視看看有多少負值。

In [11]:
len(data[data['price']<0])

124

相對於總共2000多萬筆數據而言並不多，因此我們選擇過濾這些不合理的值。

In [12]:
data = data[data['price']>= 0]

檢查缺失值。

In [13]:
na = data.isna().sum()
na

event_time              0
event_type              0
product_id              0
category_id             0
category_code    19229009
brand             8258403
price                   0
user_id                 0
dtype: int64

可以看見category_code, brand, user session中都有不少NaN的值，來看一下這三欄中有多少比例的數據是空值。

In [14]:
round(na /data.shape[0], 4)*100

event_time       0.000
event_type       0.000
product_id       0.000
category_id      0.000
category_code   98.250
brand           42.200
price            0.000
user_id          0.000
dtype: float64

category_code中NaN的比例高達98%，加上有意義相近的category_id可以作分析品類使用，因此這邊選擇直接棄用此欄位。

In [15]:
data.drop('category_code', axis= 1, inplace= True)

brand中有約42%是NaN，將他們替換成Unknown。  

In [16]:
data['brand'] = data['brand'].fillna('Unknown')

## Changing Datatype
新增時間資料與修改資料結構。

In [17]:
data['event_time'] = pd.to_datetime(data['event_time'], format= '%Y-%m-%d %H:%M:%S UTC')
data = data.sort_values(by= 'event_time') 
data['event_time'].head()

8169123   2019-10-01 00:00:00
8169124   2019-10-01 00:00:03
8169126   2019-10-01 00:00:07
8169125   2019-10-01 00:00:07
8169127   2019-10-01 00:00:15
Name: event_time, dtype: datetime64[ns]

In [18]:
data['date'] = data['event_time'].dt.date
data['month'] = data['event_time'].dt.strftime('%b %Y') #月份顯示為 月-年 E.g. Feb 2020
data['day'] = data['event_time'].dt.day
data['weekday'] = data['event_time'].dt.day_name()  # 返回星期幾
data['hr'] = data['event_time'].dt.hour

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19571767 entries, 8169123 to 16428087
Data columns (total 12 columns):
 #   Column       Dtype         
---  ------       -----         
 0   event_time   datetime64[ns]
 1   event_type   object        
 2   product_id   int64         
 3   category_id  int64         
 4   brand        object        
 5   price        float64       
 6   user_id      int64         
 7   date         object        
 8   month        object        
 9   day          int64         
 10  weekday      object        
 11  hr           int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(5)
memory usage: 1.9+ GB


## Export Data

In [20]:
data.to_csv("newdata.csv", index= False)